In [1]:
!pip install pymupdf
!pip install psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 21.1 MB/s eta 0:00:00


In [2]:
import fitz
import os

In [3]:
import requests
from bs4 import BeautifulSoup

url = "https://arxiv.org/list/cs.AI/recent"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")



In [4]:
base_url = "https://arxiv.org"
papers = []

In [5]:
for dt, dd in zip(soup.find_all("dt"), soup.find_all("dd")):
    paper_id = dt.find("a", title="Abstract")["href"]
    title = dd.find("div", class_="list-title").text.replace("Title:", "").strip()
    pdf_link = base_url + paper_id.replace("/abs/", "/pdf/")
    papers.append({ "pdf_link": pdf_link})

In [6]:
len(papers)

50

In [ ]:
papers

[{'pdf_link': 'https://arxiv.org/pdf/2505.15742'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15693'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15410'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15400'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15276'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15274'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15240'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15146'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15068'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15011'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14983'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14970'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14946'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14940'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14932'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14738'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.14689'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15810'},
 {'pdf_link': 'https://arxiv.org/pdf/2505.15808'},
 {'pdf_link': 'https://arxiv.or

In [7]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('APIKey')
from openai import OpenAI
client = OpenAI()

In [8]:
def extract_text_from_pdf(pdf_path):
        doc = fitz.open('paper.pdf')
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return text

In [9]:
database_items=[]

In [10]:
import os
import requests

os.makedirs("Summaries", exist_ok=True)
database_items = []

for paper in papers[:25]:
    url = paper['pdf_link']
    name = url.split('/')[-1]
    print(name)

    response = requests.get(url)
    with open("paper.pdf", "wb") as file:
        file.write(response.content)

    text1 = extract_text_from_pdf('/content/paper.pdf')

    # Call 1: Summary
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": "Summarize the content into 400 words with a title and a few paragraphs. Make it understandable for people with little knowledge on the field:\n\n" + text1
            }
        ],
    )
    summary_text = response.choices[0].message.content

    # Save summary
    with open(f"Summaries/{name}.txt", "w+", encoding="utf-8") as f:
        f.write(summary_text + "\n\n")

    # Call 2: Title only
    title_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": "Give a clear and concise title for this academic paper:\n\n" + text1
            }
        ],
    )
    title = title_response.choices[0].message.content.strip().replace('\n', ' ')

    # Add to database_items
    database_items.append((paper['pdf_link'], f"{name}.txt", title))


2506.09050
2506.09049
2506.09038
2506.08970
2506.08963
2506.08957
2506.08898
2506.08872
2506.08800
2506.08771
2506.08747
2506.08745
2506.08630
2506.08627
2506.08580
2506.08532
2506.08518
2506.08486
2506.08462
2506.08446
2506.08424
2506.08422
2506.08401
2506.08399
2506.08390


In [ ]:
# with open("ur1.txt", "r", encoding="utf-8") as f:
#     content = f.read()
#     # print(content)


FileNotFoundError: [Errno 2] No such file or directory: 'ur1.txt'

In [11]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )


In [12]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=userdata.get('GoogleCred')

In [13]:
def upload_all_summaries():
    summaries_folder = "Summaries"
    bucket_name = "neural_nuggets_summaries"

    # Walk through the summaries folder
    for root, dirs, files in os.walk(summaries_folder):
        for file in files:
            if file.endswith(".txt"):
                local_path = os.path.join(root, file)
                blob_name = file  # Same as the file name

                # Call your upload function
                upload_blob(bucket_name, local_path, blob_name)

# Call the function
upload_all_summaries()

File Summaries/2506.08532.txt uploaded to 2506.08532.txt.
File Summaries/2506.08462.txt uploaded to 2506.08462.txt.
File Summaries/2506.08771.txt uploaded to 2506.08771.txt.
File Summaries/2506.08745.txt uploaded to 2506.08745.txt.
File Summaries/2506.08390.txt uploaded to 2506.08390.txt.
File Summaries/2506.08627.txt uploaded to 2506.08627.txt.
File Summaries/2506.08399.txt uploaded to 2506.08399.txt.
File Summaries/2506.08580.txt uploaded to 2506.08580.txt.
File Summaries/2506.08800.txt uploaded to 2506.08800.txt.
File Summaries/2506.08963.txt uploaded to 2506.08963.txt.
File Summaries/2506.09050.txt uploaded to 2506.09050.txt.
File Summaries/2506.08970.txt uploaded to 2506.08970.txt.
File Summaries/2506.09049.txt uploaded to 2506.09049.txt.
File Summaries/2506.08747.txt uploaded to 2506.08747.txt.
File Summaries/2506.08898.txt uploaded to 2506.08898.txt.
File Summaries/2506.08422.txt uploaded to 2506.08422.txt.
File Summaries/2506.08424.txt uploaded to 2506.08424.txt.
File Summaries

In [14]:
import psycopg2

connection_string = userdata.get("neonCred")

In [21]:
def upload_todatabse(connection_string, link_summary_list):
    sql = """
    INSERT INTO neural_nuggets (pdflink, summary_path, title)
    VALUES (%s, %s, %s)
    """

    try:
        with psycopg2.connect(connection_string) as conn:
            with conn.cursor() as cur:
                cur.executemany(sql, link_summary_list)
            conn.commit()
    except Exception as e:
        print("Error during database upload:", e)

In [22]:

upload_todatabse(connection_string, database_items)

In [23]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open('paper.pdf')
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

In [24]:
text1=extract_text_from_pdf('/content/paper.pdf')


In [25]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('APIKey')

In [21]:
# from openai import OpenAI
# client = OpenAI()
# response = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "user", "content": "Summarize the content into 200 words with a title. Make it understandable for people with little knowledge on the field:\n\n"+text1}
#     ],
# )

# print(response.choices[0].message.content)


KeyboardInterrupt: 